# Titanic Kaggle Competition


In this notebook we're going to predict if a certain passenger of the titanic surived or not using machine learning.

This is a competition from kaggle, who provides the data to train, test and compare a submission

Data dictionary 

**PassengerId:**

**Pclass:** Ticket class, 1 = 1st, 2 = 2nd, 3 = 3rd

**Name:** Name of the passenger

**Sex:** Sex

**Age:** Age

**SibSp:** Number of siblings or spouses aboard the Titanic

**Parch:** Number of parents aboard the Titanic

**Ticket:** Ticket number

**Fare:** Passenger fare

**Cabin:** Cabin number

**Embarked:** Port of embarkation, C = Cherbourg, Q = Queenstown, S = Southampton

**Surived:** Did the passanger survived, this is the label, 0 = No, 1 = Yes

In [2]:
# Import the necessary libraries
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [6]:
# Read the data 
df = pd.read_csv('data/train.csv')

In [7]:
# Check the data
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [ ]:
# Let;s 